In [2]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
A = [2 0; 1 3] # fluides demandés par commande
B = [2 1; 1 2; 2 1] #Stocks de fluides par magasin
C = [1 1; 2 3; 3 2] #Couts unitaires par entrepôt
D = [1 0 0; 0 2 1] #Couts d'expédition d'un colis

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, M[1:3,1:2,1:2] >= 0)
@variable(model, Z[1:3,1:2], Bin)

# define objective function
@objective(model, Min, sum((M[:,:,1] + M[:,:,2]).*C + transpose(D).*Z))

# define constraints
for i in 1:3
    for j in 1:2
        @constraint(model, M[i,j,1] + M[i,j,2] - B[i,j] <= 0)
        @constraint(model, M[i,j,1] in MOI.Integer())
        @constraint(model, M[i,j,2] in MOI.Integer())
    end
end

#Bonne répartition des fluides 1 et 2 sur les magasins
for i in 1:2
    for j in 1:2
        @constraint(model, sum(M[:,j,i]) == A[i,j])
    end
end

# La matrice Z est binaire.
# Si Z[i,j] = 1, le magasin i va expédier un colier au demandeur j, sinon Z[i,j] = 0
for i in 1:3
    for j in 1:2
        @constraint(model, sum(M[i,:,j])/(sum(A[j,:])) <= Z[i,j])
        @constraint(model,Z[i,j] <= sum(M[i,:,j]))
    end
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 11.25 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 3 substitutions
Cgl0004I processed model has 14 rows, 13 columns (13 integer (10 of which binary)) and 37 elements
Cbc0012I Integer solution of 14 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 14, took 1 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 3 variables fixed on reduced cost
Cuts at root node changed objective from 13.5 to 14
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 s

In [3]:
JuMP.value.(Z)

3×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0
 0.0  1.0